# **Giới thiệu**

Hiện nay, với độ biến động cao của thị trường tài chính Việt Nam, việc quản lý rủi ro trở nên cực kỳ quan trọng, đặc biệt với các cổ phiếu thuộc nhóm VN30. Bằng cách sử dụng tỷ lệ Sharpe để tối ưu hóa tỷ trọng cổ phiếu và kết hợp các mô hình VaR (Value at Risk) và CVaR (Conditional Value at Risk) nhằm đánh giá tổn thất cực đoan, nhà đầu tư có thể tối ưu hóa lợi nhuận kỳ vọng so với mức độ rủi ro chấp nhận được. Tỷ lệ Sharpe giúp cân bằng lợi nhuận kỳ vọng với rủi ro, trong khi VaR và CVaR cung cấp cái nhìn sâu hơn về các kịch bản thua lỗ lớn trong điều kiện thị trường biến động mạnh. Các công cụ này không chỉ đáp ứng nhu cầu của nhà đầu tư trong thời điểm hiện tại mà còn giúp tối ưu hóa danh mục đầu tư một cách bền vững và bảo vệ khỏi những tổn thất bất ngờ, đảm bảo khả năng đưa ra quyết định đầu tư hợp lý hơn khi đối mặt với sự bất định của thị trường.

# **Phạm vi**

-Về phạm vi không gian : Dữ liệu được lấy dữ liệu giao dịch hằng ngày 

-Về phạm vi thời gian : Dữ liệu được lấy trong vòng 247 ngày gần nhất (là tổng số ngày giao dịch của VN)

# **Mục tiêu**

1.Xây dựng mô hình phân bổ danh mục tối ưu: Xác định tỷ trọng cổ phiếu tối ưu trong nhóm VN30 dựa trên tỷ lệ Sharpe nhằm tối đa hóa lợi nhuận kỳ vọng so với rủi ro.

2.Phân tích và đo lường rủi ro: Sử dụng mô hình VaR và CVaR để ước lượng mức lỗ tiềm năng và lỗ cực đoan của danh mục trong các điều kiện thị trường khác nhau.

3.So sánh hiệu quả của danh mục: Đánh giá hiệu quả của danh mục tối ưu so với các chiến lược phân bổ khác, đặc biệt trong điều kiện thị trường có biến động mạnh.

4.Đưa ra khuyến nghị quản trị rủi ro: Đề xuất các biện pháp quản trị rủi ro phù hợp cho nhà đầu tư nhằm giảm thiểu khả năng lỗ vượt quá mức cho phép trong bối cảnh thị trường hiện tại.

# **Các thư viện cần thiết cho thu thập dữ liệu** 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# **Lấy tên các công ty cần thu thập dữ liệu**

In [45]:
vnurl = pd.read_excel(r'D:\Python\SharpeRatio\vnurl.xlsx')

# ** Hàm cào dữ liệu giao dịch hằng ngày từ cafef**

In [27]:
def crawl_data(pagenumber,vnurl):
    df_final = pd.DataFrame()
    driver = webdriver.Chrome()  # Mở Chorme
    # Khởi tạo trình duyệt với Selenium
    for name in range(len(vnurl)):
        columsname = ["Ngày","Đóng cửa","Điều chỉnh","Thay đổi","Khối lượngKL","Giá trịKL(tỷ VND)","Khối lượngTT","Giá trịTT(tỷ VND)","Mở cửa","Cao nhất","Thấp nhất"]
        df_raw = pd.DataFrame(columns=columsname)
        stockname = vnurl.iloc[name,0].lower() # Tạo ra chữ tên mã CK viết thương
        driver.get(f"https://s.cafef.vn/lich-su-giao-dich-{stockname}-1.chn#data")  # URL của trang 
    
        for page in range(2,pagenumber):
            soup = BeautifulSoup(driver.page_source, "html.parser") # lấy soup   
            items = []
            for div in soup.find_all("div", class_="wrapper-table-information-owner"):
                rows = div.find_all('tr')  # Tìm tất cả các 'tr' trong mỗi 'div'
                items.extend(rows)   
            for index ,row in enumerate(items[2:]):
                    df_content = []
                    for cell in row.find_all('td')[:11]:
                        df_content.append(cell.text.strip())
                    if index % 1 == 0:
                        df_raw.loc[len(df_raw)] = df_content
            next_page_button = driver.find_element("xpath", f"//div[@class='pagination-item ' and @onclick='ownerCDL.handleChangePage({page})']")
            next_page_button.click()
            time.sleep(1)  # Dừng lại 1 giây trước khi nhấp vào trang tiếp theo  

        df_raw['Thay đổi'] = df_raw['Thay đổi'].str.extract(r'([-+]?\d+\.\d+)\s*%\)') #Chỉ lấy phần thay đổi %
        df_raw['Thay đổi'] = df_raw['Thay đổi'].astype(float)
        namestock = vnurl.iloc[[name],0].item()
        df_new = pd.DataFrame({namestock:df_raw.iloc[:,3]}) #Tạo DF mới với tên mã CK và lịch sử giao dịch
        df_final = pd.concat([df_final,df_new],axis =1)
        df_raw = df_raw.fillna(0) # Bỏ những ngày trống vào T7,CN
    return df_final

In [49]:
df_final = crawl_data(16,vnurl)

In [53]:
df_final.describe()

,ACB,BCM,BID,BVH,CTG,FPT,GAS,GVR,HDB,HPG,...,TCB,TPB,VCB,VHM,VIB,VIC,VJC,VNM,VPB,VRE
count,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,...,281.000000,281.000000,281.000000,281.000000,281.000000,281.000000,260.000000,281.000000,281.000000,281.000000
mean,0.103381,-0.006904,0.024662,0.001708,0.045658,0.175409,-0.046299,0.150961,0.160463,0.019466,...,0.134448,0.037189,0.028683,-0.052313,0.039431,-0.079431,0.032923,-0.043345,0.019964,-0.143986
std,1.218076,1.859242,1.665344,1.368518,1.755893,1.575488,1.231496,2.746031,1.473897,1.528747,...,1.693978,1.652848,1.086782,1.819349,1.473552,1.684753,1.395620,1.209004,1.479170,2.159011
min,-4.110000,-6.990000,-9.650000,-5.250000,-10.540000,-4.510000,-6.090000,-6.960000,-6.130000,-5.340000,...,-6.330000,-6.420000,-3.020000,-6.900000,-6.160000,-7.000000,-4.680000,-3.380000,-5.580000,-6.990000
25%,-0.600000,-1.010000,-0.840000,-0.700000,-0.800000,-0.780000,-0.590000,-1.100000,-0.600000,-0.820000,...,-0.840000,-0.820000,-0.560000,-0.820000,-0.720000,-0.720000,-0.490000,-0.740000,-0.780000,-1.440000
50%,0.000000,0.000000,0.000000,0.000000,0.140000,0.100000,-0.120000,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.140000,0.000000,-0.210000
75%,0.650000,0.840000,0.970000,0.620000,0.920000,0.980000,0.500000,1.550000,0.860000,0.870000,...,1.010000,0.840000,0.570000,0.710000,0.790000,0.490000,0.480000,0.540000,0.810000,0.890000
max,5.900000,6.970000,4.950000,6.950000,6.940000,6.950000,4.310000,6.990000,5.750000,5.430000,...,6.620000,6.730000,6.920000,6.900000,6.790000,7.000000,6.980000,5.760000,6.010000,6.990000


In [54]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 0 to 280
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ACB     281 non-null    float64
 1   BCM     281 non-null    float64
 2   BID     281 non-null    float64
 3   BVH     281 non-null    float64
 4   CTG     281 non-null    float64
 5   FPT     281 non-null    float64
 6   GAS     281 non-null    float64
 7   GVR     281 non-null    float64
 8   HDB     281 non-null    float64
 9   HPG     281 non-null    float64
 10  MBB     281 non-null    float64
 11  MSN     281 non-null    float64
 12  MWG     281 non-null    float64
 13  PLX     281 non-null    float64
 14  POW     281 non-null    float64
 15  SAB     281 non-null    float64
 16  SHB     281 non-null    float64
 17  SSB     281 non-null    float64
 18  SSI     281 non-null    float64
 19  STB     281 non-null    float64
 20  TCB     281 non-null    float64
 21  TPB     281 non-null    float64
 22  VC

In [55]:
df_final.dropna()

,ACB,BCM,BID,BVH,CTG,FPT,GAS,GVR,HDB,HPG,...,TCB,TPB,VCB,VHM,VIB,VIC,VJC,VNM,VPB,VRE
0,-1.97,-0.60,0.00,1.40,-0.28,-0.96,-0.43,-1.82,-1.30,-1.30,...,-0.84,-1.45,-0.11,0.00,-0.53,-0.84,-0.19,-0.45,-1.47,-0.56
1,1.20,1.05,0.10,0.12,2.73,0.00,-0.14,0.61,-1.10,-0.37,...,-0.42,0.29,2.07,0.85,-0.26,1.34,0.48,-0.30,0.99,-1.92
2,-0.40,-0.60,0.32,-0.69,-0.57,0.37,-0.28,-0.76,-0.37,-0.18,...,1.05,0.88,-0.33,-3.74,1.33,-0.85,-0.28,-1.04,0.00,0.28
3,0.20,2.14,0.11,0.12,0.72,0.37,0.14,1.38,2.44,1.12,...,0.42,-0.58,0.00,0.12,2.74,-0.24,0.29,-0.30,0.75,0.28
4,1.00,-0.30,0.00,-0.35,0.29,0.67,-0.28,0.78,-0.93,1.13,...,0.64,0.88,0.22,-2.62,0.00,-0.24,-0.10,-1.18,0.00,-0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,-3.65,-4.79,-0.49,-4.41,-2.24,-4.51,-6.09,-6.83,-2.86,-5.20,...,-4.49,-4.48,-1.52,-6.90,-5.19,-6.94,1.73,-2.90,-3.81,-6.99
256,0.00,-1.26,-1.33,0.25,-1.69,-1.20,-0.99,0.24,-0.28,0.21,...,-0.48,-0.30,-0.23,0.67,-1.08,2.88,0.00,-1.85,-0.24,-0.56
257,0.46,-0.31,2.48,0.62,0.68,1.10,0.74,3.54,0.29,0.00,...,1.46,2.13,1.78,0.22,2.49,0.12,0.41,0.14,0.00,2.31
258,-0.46,-1.55,-0.49,-1.22,-0.34,-2.05,-3.12,-3.66,-1.41,-1.44,...,-0.64,-0.30,-0.35,0.00,-1.63,0.12,-1.71,-2.90,-2.55,-2.26


In [50]:
df_final.to_excel(r'D:\Python\SharpeRatio\output1111.xlsx',index = False)

# **Thư viện cho việc xây dựng mô hình**

In [34]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
returns_df = df_final.iloc[:247, :]

# **Hàm tính lợi nhuận, rủi ro và tỷ lệ Sharpe của danh mục**

In [35]:
# Hàm tính lợi nhuận, rủi ro và tỷ lệ Sharpe của danh mục
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate):
    portfolio_return = np.sum(weights * mean_returns) * 247  # Lợi nhuận kỳ vọng hàng năm
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 247, weights)))  # Rủi ro hàng năm(Độ lệch chuẩn)
    sharpe_ratio = (portfolio_return - risk_free_rate * 247) / portfolio_std  # Tỷ lệ Sharpe
    return  sharpe_ratio

# **Hàm tính mô hình Var mô hình**

In [36]:
def calculate_var(weights, mean_returns, cov_matrix, confidence_level=0.95):
    portfolio_mean = np.sum(weights * mean_returns) * 247
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 247, weights)))
    
    # VaR theo phân phối chuẩn
    from scipy.stats import norm
    var = norm.ppf(1 - confidence_level) * portfolio_std - portfolio_mean
    return -var  # Giá trị dương để dễ diễn giải

# **Hàm tính Expected Shorted**

In [37]:
def calculate_es(weights, mean_returns, cov_matrix, confidence_level=0.95):
    portfolio_mean = np.sum(weights * mean_returns) * 247
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 247, weights)))
    
    # ES theo phân phối chuẩn
    from scipy.stats import norm
    alpha = 1 - confidence_level
    es = portfolio_mean - (portfolio_std * norm.pdf(norm.ppf(alpha)) / alpha)
    return -es  # Giá trị dương để dễ diễn giải

# **Hàm mục tiêu**

In [38]:
# Hàm mục tiêu (tối thiểu hóa -1 * tỷ lệ Sharpe)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
     return  -portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)

# **Hàm giới hạn**

In [40]:
def check_sum(weights):
    return np.sum(weights) - 1

# **Tính tỷ trọng tối ưu**

In [ ]:

# Kết quả tối ưu
optimal_weights = opt_result.x
portfolio_return =  np.sum(optimal_weights * mean_returns) * 247
portfolio_std =  portfolio_std = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix * 247, optimal_weights)))
portfolio_sharpe = portfolio_performance(optimal_weights, mean_returns, cov_matrix, risk_free_rate)
portfolio_var = calculate_var(optimal_weights, mean_returns, cov_matrix, confidence_level=0.95)
portfolio_es = calculate_es(optimal_weights, mean_returns, cov_matrix, confidence_level=0.95)

# **Hàm tổng hợp các hàm phân tích**

In [ ]:
def final(min_weight,max_weight):
    returns_df = df_final.iloc[:247, :]
    # Tính lợi nhuận trung bình và ma trận hiệp phương sai
    mean_returns = returns_df.mean()  # Lợi nhuận trung bình hàng ngày của các tài sản
    cov_matrix = returns_df.cov()     # Ma trận hiệp phương sai giữa các tài sản
    risk_free_rate = 0.04 / 247       # Lãi suất phi rủi ro (Ls trái phiếu chính phủ 10 năm ở VN 4%/năm) ,247 là số ngày giao dịch ở VN
    # Giới hạn (trọng số trong khoảng từ min đến max )
    bounds = tuple((min_weight, max_weight) for asset in range(len(mean_returns)))

    # Trọng số ban đầu (phân bổ đều)
    initial_weights = np.ones(len(mean_returns)) / len(mean_returns)

    # Tối ưu hóa tỷ lệ Sharpe
    constraints = {'type': 'eq', 'fun': check_sum}
    opt_result = minimize(neg_sharpe_ratio, initial_weights, args=(mean_returns, cov_matrix, risk_free_rate),
                        method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Kết quả tối ưu
    optimal_weights = opt_result.x
    portfolio_return =  np.sum(optimal_weights * mean_returns) * 247
    portfolio_std =  portfolio_std = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix * 247, optimal_weights)))
    portfolio_sharpe = portfolio_performance(optimal_weights, mean_returns, cov_matrix, risk_free_rate)
    portfolio_var = calculate_var(optimal_weights, mean_returns, cov_matrix, confidence_level=0.95)
    portfolio_es = calculate_es(optimal_weights, mean_returns, cov_matrix, confidence_level=0.95)

    return  optimal_weights,portfolio_return, portfolio_std,portfolio_sharpe,portfolio_var,portfolio_es
    

In [57]:
optimal_weights,portfolio_return, portfolio_std,portfolio_sharpe,portfolio_var,portfolio_es = final(0.02,0.12)
# In kết quả
print("Optimal Weights: ", optimal_weights)
print("Expected Portfolio Return: ", portfolio_return)
print("Expected Portfolio Volatility (Risk): ", portfolio_std)
print("Optimal Sharpe Ratio: ", portfolio_sharpe)
print("Value at Risk (VaR)",portfolio_var)
print("Expected Shortfall (ES) ",portfolio_es)

Optimal Weights:  [0.02       0.02       0.02       0.02       0.02       0.12
 0.02       0.03289654 0.10710346 0.02       0.02       0.02
 0.12       0.02       0.02       0.02       0.02       0.02
 0.02       0.02       0.12       0.02       0.02       0.02
 0.02       0.02       0.02       0.02       0.02       0.02      ]
Expected Portfolio Return:  32.378522585083395
Expected Portfolio Volatility (Risk):  15.539651521837182
Optimal Sharpe Ratio:  2.081032675645236
Value at Risk (VaR) 57.93897475233925
Expected Shortfall (ES)  -0.3246843667875794


In [60]:
assets = ['ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR',
       'HDB', 'HPG', 'MBB', 'MSN', 'MWG', 'PLX', 'POW', 'SAB', 'SHB', 'SSB',
       'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE']

optimal_weights_df = pd.DataFrame({'Assets':assets,'Optimal Weight(%)':optimal_weights*100})

optimal_weights_df 

,Assets,Optimal Weight(%)
0,ACB,2.000000
1,BCM,2.000000
2,BID,2.000000
3,BVH,2.000000
4,CTG,2.000000
5,FPT,12.000000
6,GAS,2.000000
7,GVR,3.289654
8,HDB,10.710346
9,HPG,2.000000
